# Data exploration

## 15-per-hour-seattle

In [71]:
import pandas as pd

comments_df = pd.read_csv('15-per-hour-seattle/comments.csv')
pvotes_df = pd.read_csv('15-per-hour-seattle/participants-votes.csv')
stathist_df = pd.read_csv('15-per-hour-seattle/stats-history.csv')
summary = pd.read_csv('15-per-hour-seattle/summary.csv', index_col=0).squeeze()
votes_df = pd.read_csv('15-per-hour-seattle/votes.csv')

In [72]:
print(f'The discussion topic is: {summary.loc['conversation-description']}')
summary

The discussion topic is: How do you think the new minimum wage law will affect Seattle? Will it be for the better or for the worse? Why?


topic
url                                                      https://pol.is/2demo
views                                                                    6184
voters                                                                    339
voters-in-conv                                                            339
commenters                                                                 33
comments                                                                   54
groups                                                                      2
conversation-description    How do you think the new minimum wage law will...
Name: $15/hour, dtype: object

In [73]:
print(f'comments_df shape: {comments_df.shape}')
comments_df.head()

comments_df shape: (54, 8)


,timestamp,datetime,comment-id,author-id,agrees,disagrees,moderated,comment-body
0,1542406176584,Sat Nov 17 05:09:36 WIB 2018,53,6172,1,0,0,Not sure I believe any of this...
1,1403067224363,Wed Jun 18 11:53:44 WIB 2014,48,215,38,18,1,"Businesses, small or large, shouldn't be able ..."
2,1403058993627,Wed Jun 18 09:36:33 WIB 2014,36,85,53,30,1,It's just going to speed up the adoption of ro...
3,1403058892294,Wed Jun 18 09:34:52 WIB 2014,20,33,43,46,1,Reactionary liberal trash. Doesn't solve the p...
4,1403054231140,Wed Jun 18 08:17:11 WIB 2014,4,0,60,25,1,While the arguments for the raise in the minim...


In [74]:
print(f'participants-votes: {pvotes_df.shape}')
pvotes_df.head()

participants-votes: (339, 60)


,participant,group-id,n-comments,n-votes,n-agree,n-disagree,0,1,2,3,...,44,45,46,47,48,49,50,51,52,53
0,0,0.0,13,13,0,0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,13,8,4,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1.0,0,14,8,6,1.0,-1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1.0,0,16,9,7,1.0,NaN,NaN,1.0,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,0.0,0,20,4,10,-1.0,0.0,-1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
print(f'stathist_df shape: {stathist_df.shape}')
stathist_df.head()

stathist_df shape: (9233, 5)


,n-votes,n-comments,n-visitors,n-voters,n-commenters
0,0,0,1,0,0
1,0,1,1,0,1
2,1,1,1,1,1
3,1,2,1,1,2
4,2,2,1,2,2


In [76]:
print(f'votes_df shape: {votes_df.shape}')
votes_df.head()

votes_df shape: (2995, 5)


,timestamp,datetime,comment-id,voter-id,vote
0,1403054214196,Wed Jun 18 08:16:54 WIB 2014,0,0,0
1,1403057082753,Wed Jun 18 09:04:42 WIB 2014,0,1,1
2,1403058957160,Wed Jun 18 09:35:57 WIB 2014,0,3,1
3,1403058913597,Wed Jun 18 09:35:13 WIB 2014,0,4,1
4,1403058827830,Wed Jun 18 09:33:47 WIB 2014,0,5,-1


### Exp 1

In [77]:
pvotes_df = pvotes_df.set_index('participant')
comments_df = comments_df.set_index('comment-id')

In [78]:
pvotes_df.head()

,group-id,n-comments,n-votes,n-agree,n-disagree,0,1,2,3,4,...,44,45,46,47,48,49,50,51,52,53
participant,,,,,,,,,,,,,,,,,,,,,
0,0.0,13,13,0,0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,0,13,8,4,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,0,14,8,6,1.0,-1.0,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,0,16,9,7,1.0,NaN,NaN,1.0,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.0,0,20,4,10,-1.0,0.0,-1.0,-1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
# Polis opinion grouping algorithm disregards participants that cast less than 7 votes. 
# See: https://github.com/compdemocracy/notebooks/blob/master/American%20Assembly%20BG%20analysis%20v0.8%20subconversation%2C%20every%20vote.ipynb
print(f'There are {pvotes_df[pd.isnull(pvotes_df['group-id'])].shape[0]} participants that casted less than 7 votes.')

# In our exercise, we will have to disregard them as well.
pvcounted_df = pvotes_df[pvotes_df['n-votes'] >= 7]
print(f'There are {pvcounted_df.shape[0]} participants left in the analysis.')

There are 201 participants that casted less than 7 votes.
There are 138 participants left in the analysis.


In [80]:
metadata_cols = ['group-id', 'n-comments', 'n-votes', 'n-agree', 'n-disagree']
metadata = pvcounted_df[metadata_cols]
metadata.head()

,group-id,n-comments,n-votes,n-agree,n-disagree
participant,,,,,
0,0.0,13,13,0,0
1,1.0,0,13,8,4
3,1.0,0,14,8,6
4,1.0,0,16,9,7
5,0.0,0,20,4,10


In [82]:
pc_matrix = pvcounted_df.drop(columns=metadata_cols)
pc_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
participant,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,-1.0,NaN,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,NaN,NaN,1.0,NaN,1.0,1.0,1.0,-1.0,1.0,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-1.0,0.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
print(f'After taking out the null participants, there are {metadata['n-votes'].sum()} votes counted.')
print(f'There are {pc_matrix.isnull().sum().sum()} null votes in the participant-comment matrix')

After taking out the null participants, there are 2371 votes counted.
There are 5081 null votes in the participant-comment matrix


In [95]:
# We fill the null votes with 0s. 
pc_matrix = pc_matrix.fillna(0)
print(f'After filling the null votes with 0s, there are {pc_matrix.isnull().sum().sum()} null votes in the participant-comment matrix')

After filling the null votes with 0s, there are 0 null votes in the participant-comment matrix


In [ ]:
# Perform k-means clustering on pc_matrix
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns

# Choose the number of clusters (e.g., 3)
range_n_clusters = range(2,7)

silhouette_scores = []

for n_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
    # fit_predict() directly returns the labels
    # fit() doesn't return the labels; one has to call kmeans.labels_ to get the labels
    cluster_labels = kmeans.fit_predict(pc_matrix)

    silhouette_avg = silhouette_score(pc_matrix, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    print(f'For n_clusters = {n_clusters}, the silhouette score is: {silhouette_avg}')

For n_clusters = 2, the silhouette score is: 0.11076700476513324
For n_clusters = 3, the silhouette score is: 0.0866795971231056
For n_clusters = 4, the silhouette score is: 0.06928869192127501
For n_clusters = 5, the silhouette score is: 0.05467453921317674
For n_clusters = 6, the silhouette score is: 0.06638218922010929


In [ ]:
# Given the above silhouette scores, the number of clusters should be 2.
n_clusters = 2

# Initialize the KMeans model
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')  # explicitly set n_init

# Fit the model to the data
kmeans.fit(pc_matrix)

# Get the cluster labels
cluster_labels = kmeans.labels_

# Add the cluster labels to the metadata DataFrame
metadata.loc[:, 'cluster'] = cluster_labels

print(metadata.head())

             group-id  n-comments  n-votes  n-agree  n-disagree  cluster
participant                                                             
0                 0.0          13       13        0           0        1
1                 1.0           0       13        8           4        1
3                 1.0           0       14        8           6        1
4                 1.0           0       16        9           7        1
5                 0.0           0       20        4          10        0


In [105]:
# Compare the group-id (given by the original source) with the cluster (calculated by us)
metadata.loc[:, 'cluster'] = metadata['cluster'].astype(float)

diff_clustering = metadata[metadata['group-id'] != metadata['cluster']]
print(f'There are {diff_clustering.shape[0]} participants that were assigned to a different cluster than the one they were assigned to by the original source.')
diff_pindex = diff_clustering.index.tolist()

There are 38 participants that were assigned to a different cluster than the one they were assigned to by the original source.


In [106]:
diff_pindex

[0,
 8,
 12,
 20,
 32,
 34,
 68,
 74,
 83,
 102,
 5806,
 5808,
 5999,
 6053,
 6056,
 6068,
 6085,
 6128,
 6154,
 6162,
 6180,
 21,
 5986,
 6031,
 6078,
 6082,
 6133,
 40,
 6123,
 6156,
 6171,
 6175,
 6176,
 6144,
 46,
 5983,
 6101,
 6169]

## American Assembly

In [54]:
import pandas as pd

comments_df = pd.read_csv('american-assembly.bowling-green/comments.csv')
pvotes_df = pd.read_csv('american-assembly.bowling-green/participants-votes.csv')
stathist_df = pd.read_csv('american-assembly.bowling-green/stats-history.csv')
summary = pd.read_csv('american-assembly.bowling-green/summary.csv', index_col=0).squeeze()
votes_df = pd.read_csv('american-assembly.bowling-green/votes.csv')

## NH-democratic-ai-2

In [51]:
comments_df = pd.read_csv('NH-democratic-ai-2/comments.csv')
pvotes_df = pd.read_csv('NH-democratic-ai-2/participant-votes.csv')
summary = pd.read_csv('NH-democratic-ai-2/summary.csv', index_col=0).squeeze()
votes_df = pd.read_csv('NH-democratic-ai-2/votes.csv')
comgroup_df = pd.read_csv('NH-democratic-ai-2/comment-groups.csv')
comgroup_df

,comment-id,comment,total-votes,total-agrees,total-disagrees,total-passes,group-a-votes,group-a-agrees,group-a-disagrees,group-a-passes,group-b-votes,group-b-agrees,group-b-disagrees,group-b-passes,group-c-votes,group-c-agrees,group-c-disagrees,group-c-passes
0,1,"Even if AI-generated works are legal, there’s ...",27,19,6,2,19,16,1,2,5,3,2,0,3,0,3,0
1,2,"AI-generated content will flood the internet, ...",27,13,11,3,19,9,7,3,5,1,4,0,3,3,0,0
2,3,"If AI replaces human artists, it will kill the...",8,3,5,0,4,2,2,0,3,1,2,0,1,0,1,0
3,4,Small creators will lose their livelihoods if ...,24,12,8,4,17,10,5,2,5,2,2,1,2,0,1,1
4,5,Big AI companies are profiting off artists’ wo...,22,15,4,3,15,12,0,3,4,0,4,0,3,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,57,Bernie for president,5,2,0,3,4,2,0,2,1,0,0,1,0,0,0,0
57,58,"It's impractical to add copyright laws to AI, ...",3,2,0,1,2,1,0,1,1,1,0,0,0,0,0,0
58,59,We need a balanced approach that does not guar...,2,2,0,0,1,1,0,0,1,1,0,0,0,0,0,0
59,60,"Soon, pretty much everything is at least part-...",3,1,1,1,2,1,0,1,1,0,1,0,0,0,0,0


In [52]:
pvotes_df['group-id'].unique()

array([0, 2, 1])